### Feature Selection

- Filtering 방법  
=> 타겟/ 종속변수에 관련성이 높은 피쳐/독립변수 선택  
=> 상관계수  
=> 몇 개? K개 => 점진적으로 늘려가기  

- Wrapper 방법  
=> Forward 방식 : 기준에 따라 K개 피쳐를 선택 -> 성능평가 -> 피쳐추가  
=> Backward 방식 : 모든 피쳐 선택 -> 성능평가 -> 피쳐 제거  
=> Step-Wide 방식 : Forward + Backward  

- Embedding 방법  
=> 모델 내부에 피쳐 선택 기능 포함된 것 -> important Feature, 주성분분석(비지도 학습)  

### Tunning

- 모델의 성능 높이기  
=> scikit-learn에서 모델 인스턴스의 score 메서드  
=> 모델을 평가 기준이 되는 성능 지표 => 분류/회귀에 따라 다름  

- 모델 성능에 영향을 미치는 매개변수 ==> Hyper-parameter  
=> KNN -> K 개수  
=> LinearRegression(coef_,intercept_) -> 가중치, 기울기 초기화 값, 규제(릿찌,라쏘...)  
=> 학습 알고리즘마다 Hyper-parameter 다름!  

- 피쳐 제어  
=> 피쳐 개수 조절  
=> 피쳐 추출 / 압축  

### AutoML
- 자동으로 Hyper-parameter 찾아주지만 유료임..

### 기계학습
- 데이터 기반 -> 규칙/패턴 -> 수식 -> 모델

- 미래 예측
- 정보 정확 제공

- 학습용 DS ==> 학습용 + 테스트용
- 테스트용 DS 

### ML Process
1. 데이터 준비 => 데이터 로딩, 데이터 확인(실제 데이터 확인 및 탐색)
2. 데이터 전처리  
=> 정제(결측치,중복값,이상치,컬럼 고유값....)  
=> 피쳐에 대한 처리 (인코딩,스케일링...)
=> 피쳐 선택, 가공  
=> 피쳐와 타겟(독립변수와 종속변수)
3. 학습 준비  
=> 데이터 셋(train,validation,test)  
=> 데이터 부족 및 일반화 위해서 train,test 데이터 셋 분리  
4. 학습 진행  
=> 교차 검증으로 학습 진행 : train 데이터 셋 사용  
5. 모델 평가  
=> test 데이터셋으로 평가
=> 평가 기준 : 분류와 회귀가 다름!
### 모델의 성능을 높이는 작업 즉, 튜닝 진행
- Hyper parameter 제어 : 모델 인스턴스 생성 시 매개변수로 설정  
- 새로운 모델로 학습을 진행 => 평가


##### 붓꽃 품종 분류
- 목표 : 붓꽃의 3개 품종을 분류하기
- 데이터셋 : 내장 데이터셋
- 피쳐 : 4개
- 타겟 : 품종 1개
- 학습 : 지도학습 > 분류

[1] 데이터 준비

In [780]:
# 모듈 로딩
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
import numpy as np

In [781]:
# 내장 데이터셋 로딩
data = load_iris(as_frame=True)

In [782]:
# Bunch 인스턴스 => dict와 유사한 형태
data.keys()

dict_keys(['data', 'target', 'frame', 'target_names', 'DESCR', 'feature_names', 'filename', 'data_module'])

In [783]:
featureDF = data['data']
targetSR = data['target']

In [784]:
featureDF.shape, targetSR.shape

((150, 4), (150,))

In [785]:
featureDF.head(1), targetSR.head(1)

(   sepal length (cm)  sepal width (cm)  petal length (cm)  petal width (cm)
 0                5.1               3.5                1.4               0.2,
 0    0
 Name: target, dtype: int32)

[2] 학습을 위한 데이터 셋 준비 => 학습용, 검증용, 테스트용

In [786]:
# 학습용 & 테스트용 분리
X_train, X_test, y_train, y_test = train_test_split(featureDF,
                                                    targetSR,
                                                    stratify = targetSR
                                                    )

In [787]:
# 학습용 & 검증용 분리
X_train, X_val, y_train, y_val = train_test_split(X_train,
                                                    y_train,
                                                    stratify = y_train
                                                    )

In [788]:
print(f'train DS : {X_train.shape[0]}   {X_train.shape[0]/featureDF.shape[0]}%')
print(f'Val DS : {X_val.shape[0]}   {X_val.shape[0]/featureDF.shape[0]:.2f}%')
print(f'test DS : {X_test.shape[0]}   {X_test.shape[0]/featureDF.shape[0]:.2f}%')

train DS : 84   0.56%
Val DS : 28   0.19%
test DS : 38   0.25%


[3] 교차검증 방식

In [789]:
# 모듈 로딩
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.tree import DecisionTreeClassifier

In [790]:
# 모델 인스턴스 생성
dtc_model = DecisionTreeClassifier()

# [3-1] Kfold기반-------------------------------
# 정확도 저장 리스트
accuracys = []

# KFold 인스턴스 생성 [기본 k=5]
Kfold = KFold()


In [791]:
# k번만큼 학습이 진행
# -> k등분 후 학습용 데이터셋 인덱스, 검증용 데이터셋 인덱스
for idx, (train_index, val_index) in enumerate(Kfold.split(featureDF),1):

    #print(f'train_index : {train_index.tolist()}')

    # X_train, X_val 데이터셋 설정
    X_train,y_train = featureDF.iloc[train_index.tolist()], targetSR[train_index.tolist()]
    X_val,y_val = featureDF.iloc[val_index.tolist()], targetSR[val_index.tolist()]

    #학습 진행
    dtc_model.fit(X_train,y_train)

    # 평가 => 분류의 경우 score() 메서드 => 정확도 반환
    train_acc = dtc_model.score(X_train,y_train)
    val_acc = dtc_model.score(X_val,y_val)

    accuracys.append([train_acc,val_acc])
    print(f'{idx}번째 Train 정확도 : {train_acc} Val 정확도 : {val_acc}')

1번째 Train 정확도 : 1.0 Val 정확도 : 1.0
2번째 Train 정확도 : 1.0 Val 정확도 : 0.9666666666666667
3번째 Train 정확도 : 1.0 Val 정확도 : 0.8333333333333334
4번째 Train 정확도 : 1.0 Val 정확도 : 0.9333333333333333
5번째 Train 정확도 : 1.0 Val 정확도 : 0.7666666666666667


In [792]:
# 평균 계산
train_mean = sum([value[0]  for value in accuracys])/Kfold.n_splits # K의 값 하드코딩 말고 이 방법 쓰기
test_mean = sum([value[1]  for value in accuracys])/Kfold.n_splits

print(f'Train 정확도 : {train_mean}  Val 정확도 : {test_mean:.2f}')

Train 정확도 : 1.0  Val 정확도 : 0.90


[3-2] Straitified Kfold
- 정답/레이블/타겟의 비율을 고려해서 데이터를 나눠줌!!!!

In [793]:
accuracys = []

skFold=StratifiedKFold()
# k번만큼 학습이 진행
# -> k등분 후 학습용 데이터셋 인덱스, 검증용 데이터셋 인덱스
for idx, (train_index, val_index) in enumerate(skFold.split(featureDF,targetSR),1):
    #                                                              (타겟을 줘야함)

    #print(f'train_index : {train_index.tolist()}')

    # X_train, X_val 데이터셋 설정
    X_train,y_train = featureDF.iloc[train_index.tolist()], targetSR[train_index.tolist()]
    X_val,y_val = featureDF.iloc[val_index.tolist()], targetSR[val_index.tolist()]

    #학습 진행
    dtc_model.fit(X_train,y_train)

    # 평가 => 분류의 경우 score() 메서드 => 정확도 반환
    train_acc = dtc_model.score(X_train,y_train)
    val_acc = dtc_model.score(X_val,y_val)

    accuracys.append([train_acc,val_acc])
    print(f'{idx}번째 Train 정확도 : {train_acc} Val 정확도 : {val_acc}')

1번째 Train 정확도 : 1.0 Val 정확도 : 0.9666666666666667
2번째 Train 정확도 : 1.0 Val 정확도 : 0.9666666666666667
3번째 Train 정확도 : 1.0 Val 정확도 : 0.9
4번째 Train 정확도 : 1.0 Val 정확도 : 0.9333333333333333
5번째 Train 정확도 : 1.0 Val 정확도 : 1.0


In [794]:
# 평균 계산
train_mean = sum([value[0]  for value in accuracys])/skFold.n_splits # K의 값 하드코딩 말고 이 방법 쓰기
test_mean = sum([value[1]  for value in accuracys])/skFold.n_splits

print(f'Train 정확도 : {train_mean}  Val 정확도 : {test_mean:.2f}')

Train 정확도 : 1.0  Val 정확도 : 0.95


- 교차검증 및 성능평가 동시 진행 함수  
=> cross_val_score, cross_val_predict  
=> cross_validate

In [795]:
# 모듈 로딩
from sklearn.model_selection import cross_val_predict,cross_val_score,cross_validate

In [796]:
### [1] 전체 DS ==> 학습용과 테스트용 DS 분리
X_train,X_test,y_train,y_test = train_test_split(featureDF,
                                                 targetSR,
                                                 stratify = targetSR,
                                                 random_state=10)

In [797]:
### cross_val_predict
predict = cross_val_predict(dtc_model,X_train,y_train,cv=3)

In [798]:
print(f'predict : {predict}')

predict : [0 2 1 0 1 1 2 1 1 1 2 0 2 2 1 2 0 2 0 2 0 1 1 1 0 1 0 2 2 2 1 2 0 0 0 1 0
 0 0 1 1 2 0 2 1 0 2 1 0 0 2 0 2 0 0 0 1 1 1 0 1 2 1 0 1 1 2 1 2 0 1 2 1 0
 2 2 2 1 0 2 2 1 1 2 1 2 1 2 0 0 0 0 0 2 1 2 2 2 0 0 2 0 1 2 2 2 0 1 0 2 0
 1]


In [799]:
### cross_val_predict
score = cross_val_score(dtc_model,X_train,y_train)

In [800]:
print(f'score : {score}')

score : [0.95652174 0.95652174 1.         0.90909091 0.90909091]


In [801]:
### cross_validate
result = cross_validate(dtc_model,X_train,y_train)

In [802]:
result
# {'fit_time': array([0.00224161, 0.00199413, 0.0019958 , 0.00171423, 0.00100827]),
#  'score_time': array([0.00134301, 0.0011766 , 0.00099635, 0.        , 0.00277662]),
#  'test_score': array([0.96666667, 0.96666667, 0.9       , 1.        , 1.        ] => valudation score)}

{'fit_time': array([0.00199413, 0.00199246, 0.00460505, 0.        , 0.        ]),
 'score_time': array([0.00099707, 0.00199246, 0.        , 0.        , 0.        ]),
 'test_score': array([0.95652174, 0.95652174, 0.95454545, 0.90909091, 0.90909091])}

In [803]:
result = cross_validate(dtc_model,X_train,y_train,return_train_score=True,return_estimator=True)

In [804]:
resultDF=pd.DataFrame(result).loc[:,['test_score', 'train_score']]
resultDF

,test_score,train_score
0,0.956522,1.0
1,0.956522,1.0
2,1.000000,1.0
3,0.909091,1.0
4,0.909091,1.0


* 최적화된 모델 추출

In [805]:
# 반환 받은 모델 확인하기
result['estimator']

# 4번이 test와 train의 값이 가장 차이가 적게나서 최적의 모델이라고 판단
best_model = result['estimator'][0]

* 테스트 데이터로 확인

In [806]:
best_model.score(X_test,y_test)

0.9736842105263158